In [ ]:
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd
import math
import textstat
import numpy as np
import fnmatch

#nltk.install("all")

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

from seg import Segmentation

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

# os.remove(path)
# if os.path.exists(path):
#    os.remove(path)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 10 19:16:37 2020

@author: pablonicolasr
"""
from dataset import generate
from cluster import cluster_ip
from lxml import etree


actual_directory = os.getcwd()

ruta = os.path.join(actual_directory, "mini_corpus")
ruta_del_directorio = str(ruta) + "\\*.txt"
# Me produce una lista con los nombres de los archivos txt del corpus
textos_originales = glob.glob(ruta_del_directorio)


if not os.path.exists(str(actual_directory)+"\\cleaned_corpus"):
    os.mkdir(str(actual_directory)+'\\cleaned_corpus')
    
cleaned_corpus = str(actual_directory)+"\\cleaned_corpus"

# De esta manera le voy robo el nombre a los xml
path_xml = str(actual_directory) + "\\mini_corpus" + "\\*.xml"
xml_docs = glob.glob(path_xml)


if not os.path.exists(str(actual_directory)+"\\sospechoso"):
    os.mkdir(str(actual_directory)+'\\sospechoso')
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(str(actual_directory)+"\\sospechoso")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory)+"\\sospechoso", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

    
if not os.path.exists(str(actual_directory) + "\\xml\\output_xml"):
    os.mkdir(str(actual_directory) + "\\xml\\output_xml")
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(str(actual_directory) + "\\xml\\output_xml")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory) + "\\xml\\output_xml", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")


sospechosos_path = str(actual_directory) + "\\sospechoso\\"
output_xml = str(actual_directory) + "\\xml\\output_xml\\"


f = {}
# Comienzo a leer el conjunto de textos
for i in range(len(textos_originales)):
    print("File: " + str(i+1))
    f[i] = open(textos_originales[i], encoding= "utf-8-sig")
    text = f[i].read()
    f[i].close()            
    
    # Nombre de Archivo
    filename = os.path.basename(textos_originales[i])
    # Separo nombre de archivo y extension
    filename2, extension = os.path.splitext(filename)
    
    Seg = Segmentation(text)
    # La variable data_text posee el texto completo, separado en párrafos
    data_text = Seg.paraSegmentation()
    data_list = []
    length = []
    
    # Se tiene guardar el texto sin limpiar
    for segmento in data_text:
        # A cada segmento se le borra el salto de línea
        segmento = re.sub("\n", " ", segmento)
        # Se guarda la longitud del segmento
        longitud = len(segmento)
        length.append(longitud)
        # Se guarda el segmento
        data_list.append(segmento)
    
    # De aqui empezar a procesar la limpieza de texto 
    # Some code...
    # .....
    # .....
    
    rootPath = ruta + "\\"+ filename
    outputPath = str(actual_directory) + "\\csv\\"+ filename2 + ".csv"
    
    if os.path.exists(str(actual_directory) + "\\csv\\"+ filename2 + ".csv"):
        os.remove(str(actual_directory) + "\\csv\\"+ filename2 + ".csv")
    
    generate(rootPath, outputPath, filename, "english")
    
    df = pd.read_csv(outputPath)
    
    # Jugar con estos números // Tarea
    
    # samples = len(length) * 0.075
    
    # decimal, entero = math.modf(samples)
    
    # if decimal >= 0.5:
        # min_samples = entero + 1
    # else:
        # min_samples = entero
    
    # Este número fue sacado de un paper
    min_samples = 4
    
    # Normalizar de forma correcta y eficiente el indice de facilidad de lectura
    df["fleshReadingEase"] = df["fleshReadingEase"] / df["fleshReadingEase"].max()
    
    # Seleccionar los valores numéricos del df
    # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
    dataframe = df.iloc[:,[2,3,4]]
    
    # Aprendizaje No Supervisado
    cluster = cluster_ip(dataframe, min_samples, "cosine")
    labels = cluster.labels_    
    df["labels"] = labels
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum=len(set(labels))
    n_noise = list(labels).count(-1) 
    
    print("Estimated number of clusters: %d" % realClusterNum)
    print("Estimated number of noise points: %d" % n_noise)
    counts = df["labels"].value_counts().to_dict()
    print(counts)
    
    # Me sirve para guardar los casos de plagio y compararlos
    xml_name = os.path.basename(xml_docs[i])
    xml_name2, extension = os.path.splitext(xml_name)
    
    # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
    sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")
    
    # Los fragmentos que fueron clasificados como plagio
    # Se guardan en el archivo .txt
    # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
    # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
    for x in range(len(df["labels"])):
        l = df["labels"][x]
        if l == -1:
            print(data_list[x])
            print(x)
            sherlock.write(data_list[x]+os.linesep)       
               
    sherlock.close()    
    
    document_el = etree.Element("document", reference = xml_name2 + ".txt")
    
    offset = 0
    
    for y in range(len(df["labels"])):       
        
        l = df["labels"][y]
        
        if l == -1: 
            
            feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(offset), this_length=str(length[y]))
    
        offset += length[y] + 1
                          
            
    et = etree.ElementTree(document_el)
    with open(output_xml+xml_name, "wb") as fe:
        et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)